IMPORT LIBRARIES

In [19]:
import pandas as pd
import numpy as np

import scipy as sp
import sklearn as sk

import random
import time

import warnings
warnings.filterwarnings('ignore')
from subprocess import check_output


In [21]:
#common model algorithms
from sklearn import svm, tree, gaussian_process, linear_model, ensemble

#common model help tools
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

#Data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

#set visual parameters
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 16,12

LOAD DATA
--

In [4]:
#import data
train_data = pd.read_csv("C:/Users/kpunyakoti/Desktop/Data Science/Kaggle/Housing Prices/Source Data files/train.csv")
test_data = pd.read_csv("C:/Users/kpunyakoti/Desktop/Data Science/Kaggle/Housing Prices/Source Data files/test.csv")

play_data = train_data.copy() #make copy of training data to play with
cleaning_data = [play_data, test_data] #pass both data sets to clean everything at once

#print(test_data.info())
train_data.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
94,95,60,RL,69.0,9337,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,204750
1106,1107,20,RL,114.0,10357,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,179900
1444,1445,20,RL,63.0,8500,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,11,2007,WD,Normal,179600
199,200,20,RL,76.0,9591,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2009,WD,Normal,274900
1221,1222,20,RL,55.0,8250,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2008,WD,Normal,134000


4 Cs of DATA CLEANING: Correcting, Completing, Creating and Converting
---

In [5]:
#check for null values
print('Train data with null values:\n', play_data.isnull().sum())
print("-"*20)
print('Test data with null values:\n', test_data.isnull().sum())

Train data with null values:
 Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
                 ... 
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu       690
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageCars          0
GarageArea          0
GarageQual         81
Ga

In [6]:
# finding categorical variables
categorical = [var for var in play_data.columns if play_data[var].dtype=='O']
print('There are {} categorical variables'.format(len(categorical)))
print(play_data.shape)
print(test_data.shape)

There are 43 categorical variables
(1460, 81)
(1459, 80)


Treating Outliers
--

In [7]:
#impute for Garage Year Built less than Year Built - total 9 such items in both data sets #part of feature engineerinig
for data in cleaning_data:
    data.loc[((data['GarageYrBlt'] > 0) & ((data['GarageYrBlt']-data['YearBuilt']) < 0)),'GarageYrBlt'] = data['YearBuilt']

In [8]:
#outliers for GrLivArea
cond2 = ((play_data.Id == 524) | (play_data.Id == 1299))
play_data.loc[cond2,'SalePrice'] = 611657 #imputing with max saleprice for Partial SaleType
#however, in one of the kernel's these records were dropped
#play_data = play_data.drop(play_data[(play_data['GrLivArea']>4000) & (play_data['SalePrice']<300000)].index)
#play_data.drop(play_data[play_data.Id == 1299].index, inplace=True)  #outliers for GrLivArea

In [9]:
#impute for LotArea with max values for SalePriceBins - proved to be not worth
#play_data.loc[(play_data.Id == 314),'LotArea'] = 27650 #imputing with max LotArea for GrLivAreaBin
#play_data.loc[(play_data.Id == 336),'LotArea'] = 16669 #imputing with max LotArea for GrLivAreaBin
#play_data.loc[(play_data.Id == 250),'LotArea'] = 27650 #imputing with max LotArea for SalePriceBin
#play_data.loc[(play_data.Id == 707),'LotArea'] = 25350 #imputing with mean of LotArea for SalePriceBin and GrLivAreaBin

Treating Missing Values
--

In [10]:
#FILL NAs with No entities
#None Columns:
none_cols = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageFinish',
           'GarageQual','GarageCond','PoolQC','Fence','Alley','MiscFeature','MasVnrType', 'MiscFeature']
for x in none_cols:
    for data in cleaning_data:
        data[x].fillna('None', inplace = True)
#mode columns
mode_cols = ['Exterior1st','Exterior2nd','SaleType','KitchenQual','Utilities','Functional','Electrical']
for y in mode_cols:
    for data in cleaning_data:
        data[y].fillna(data[y].mode()[0], inplace = True)
#zero columns
zero_cols = ['MasVnrArea','TotalBsmtSF','GarageYrBlt','GarageCars','GarageArea','BsmtFullBath','BsmtHalfBath','BsmtFinSF1',
           'BsmtFinSF2','BsmtUnfSF']
for z in zero_cols:
    for data in cleaning_data:
        data[z].fillna(0, inplace=True)

In [11]:
#BsmtExposure-one row is marked NA(nobasement) where we have values for Basement:row-949-replace the value with mode which 'No'
cond = (play_data.Id == 949)
play_data.loc[cond,'BsmtExposure'] = 'No'

#BsmtFinType2-one row marked NA(nobasement) where we have values for Basement:row-333-replace the value with mode which is 'unf'
cond2 = (play_data.Id == 333)
play_data.loc[cond2, 'BsmtFinType2'] = 'Unf'

In [12]:
# Imputing LotFrontage with the median (middle) value
for data in cleaning_data:
    data['LotFrontage'] = data.groupby('Neighborhood')['LotFrontage'].apply(lambda x: x.fillna(x.median()))

In [13]:
#MSZoning - 4NAs -  impute using MSSubClass: where 20-MSz is RL, for 30-RM, 70-RM
cond1 = ((test_data['MSZoning'].isnull()==True) & (str(test_data['MSSubClass']) == '20'))
test_data.loc[cond1,'MSZoning'] = 'RL'
cond2 = ((test_data['MSZoning'].isnull()==True) & (str(test_data['MSSubClass']) == '30'))
test_data.loc[cond2,'MSZoning'] = 'RM'
cond3 = ((test_data['MSZoning'].isnull()==True) & (str(test_data['MSSubClass']) == '70'))
test_data.loc[cond3,'MSZoning'] = 'RM'

Feature Engineering
--

In [14]:
#Create new column Duplex
#for data in cleaning_data:
#    data.loc[(data['2ndFlrSF']==0),'Duplex'] = 'N'
 #   data.loc[(data['2ndFlrSF']>0),'Duplex'] = 'Y'
    #data.loc[((data['Foundation']=='Wood')|(data['Foundation']=='Slab')|(data['Foundation']=='Stone')),'Foundation'] = 'Other'

In [15]:
#Create Bins
for data in cleaning_data:
    data['TotalSF'] = data['TotalBsmtSF']+data['GrLivArea']
    data['TotalPorchSF'] = data['EnclosedPorch']+data['OpenPorchSF']+data['3SsnPorch']+data['ScreenPorch']

Column Drop
--

In [16]:
# DROPPING COLUMSN with more than 97% same values

drop_cols = ['Street','Utilities','Condition2', 'RoofMatl', 'Heating', 'PoolQC']   #
play_data.drop(drop_cols, axis=1, inplace=True)
test_data.drop(drop_cols, axis=1, inplace=True)

In [17]:
print(play_data.shape)
print(test_data.shape)

(1460, 77)
(1459, 76)


In [18]:
#Export Cleaned training and test datasets - Phase 1 export [After datatype conversion and null treatment]
play_data.to_csv("C:/Users/kpunyakoti/Desktop/Data Science/Kaggle/Housing Prices/Source Data files/clean_new_train_data.csv", index=False)
test_data.to_csv("C:/Users/kpunyakoti/Desktop/Data Science/Kaggle/Housing Prices/Source Data files/clean_new_test_data.csv", index=False)

Feature Scaling
--

